# Dall’istruzione al benessere: un’analisi dell’impatto educativo sui principali indicatori di sviluppo

## Obiettivo

### Valutare la relazione tra partecipazione scolastica, completamento dell’istruzione e spesa educativa, e combinare questi indicatori con alcuni parametri di sviluppo e benessere per individuare quali strategie educative siano più efficaci nel migliorare lo sviluppo umano e socio-economico in alcuni Paesi significativi.

A tale scopo mi servirò principalmente del dataset “EdStats” della World Bank, che raccoglie le principali statistiche globali sull’istruzione.

Per ampliare la prospettiva e valutare l’impatto dell’istruzione su dimensioni più ampie dello sviluppo umano, l’analisi integra inoltre tre dataset aggiuntivi della World Bank: il tasso di mortalità infantile, l’aspettativa di vita alla nascita e il GDP per capita. Questi indicatori permettono di osservare come i progressi educativi si colleghino a miglioramenti in ambito sanitario, economico e sociale.

I Paesi inclusi nell’analisi sono stati selezionati in base alla rilevanza e alla diversificazione geografica e socio-politica a livello mondiale, privilegiando quei contesti che offrono un quadro temporale sufficientemente ampio da consentire confronti significativi e affidabili, in termini di disponibilità dei dati.

Il mio intento, rispetto alla consegna, è stato dunque quello di costruire un’analisi che non si limitasse a mostrare valori puntuali,
ma che cercasse attivamente connessioni e pattern utili a riflettere sull’impatto dell’istruzione sul benessere delle società. 
Le visualizzazioni prodotte e le correlazioni calcolate mirano a fornire una prima interpretazione dei dati, 
utile come base per elaborare eventuali raccomandazioni, approfondimenti futuri o politiche mirate.

In [ ]:
# Importo le librerie necessarie

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

In [ ]:
# Carico il dataset EdStats di World Bank

df_ed = pd.read_excel('/Users/valerioquaranta/Documents/Data Manipulation and Visualization/Progetto/Prog_corr/EdStats_v01.xlsx')

In [ ]:
df_ed.head()

In [ ]:
# Visualizzo le info del DataFrame

df_ed.info()

In [ ]:
# Visualizzo i valori della colonna 'Indicator name'

df_ed['Indicator name'].unique()

In [ ]:
# Cerco gli indicatori che abbiano nel nome le parole 'enrolment' e 'primary' 

en_indicators = df_ed[df_ed['Indicator name'].str.contains('enrolment', case=False) & df_ed['Indicator name'].str.contains('primary', case=False)]
en_indicators['Indicator name'].unique()

In [ ]:
# Cerco gli indicatori che abbiano nel nome le paarole 'completion' e 'secondary' 

com_indicators = df_ed[df_ed['Indicator name'].str.contains('completion', case=False) & df_ed['Indicator name'].str.contains('secondary', case=False)]
com_indicators['Indicator name'].unique()

In [ ]:
# Cerco gli indicatori che abbiano nel nome la paorla 'GDP' 

exp_indicators = df_ed[df_ed['Indicator name'].str.contains('GDP', case=False, na=False)]
exp_indicators['Indicator name'].unique()

In [ ]:
# Pulisco i nomi delle colonne

df_ed.columns = df_ed.columns.str.strip()

In [ ]:
# Creo una lista di paesi da analizare e controllo se sono presenti nel dataframe

countries_list = ['Italy', 'Germany', 'United States', 'Canada', 'China', 'India', 'Brazil', 'South Africa', 'Kenya', 'Australia']
available_countries = df_ed['Country name'].unique()
missing_countries = [country for country in countries_list if country not in available_countries]
if missing_countries:
    print(f"I seguenti Paesi non sono presenti nel dataframe: {missing_countries}")
else:
    print("Tutti i Paesi sono presenti nel dataframe.")


In [ ]:
# Filtro il dataframe per includere una lista di Paesi significativi 

df_ed = df_ed[df_ed['Country name'].isin(countries_list)]
df_ed['Country name'].unique()

In [ ]:
# Creo una lista di indicatori da analizzare e controllo se sono presenti nel dataframe (Gli indicatori sono stati scelti dopo il filtraggio in base alla loro rilevanza per l'analisi dell'educazione)

indicators_list = ['Total net enrolment rate, primary, both sexes (%)', 
                   'Completion rate, upper secondary education, both sexes (%)',
                   'Government expenditure on education as a percentage of GDP (%)'
]

available_indicators = df_ed['Indicator name'].unique()
missing_indicators = [indicator for indicator in indicators_list if indicator not in available_indicators]
if missing_indicators:
    print(f"I seguenti indicatori non sono presenti nel dataframe: {missing_indicators}")
else: 
    print("Tutti gli indicatori sono presenti nel dataframe.")

In [ ]:
# Filtro il dataframe per includere solo gli indicatori selezionati

df_ed = df_ed[df_ed['Indicator name'].isin(indicators_list)]
df_ed['Indicator name'].unique()

In [ ]:
# Resetto l'indce e controllo il dataframe

df_ed = df_ed.reset_index(drop=True)
df_ed.head()

In [ ]:
# Converto la tabella da formato long a wide per facilitare l'analisi

df_ed = df_ed.melt(
    id_vars=['Country name', 'Country code', 'Indicator name', 'Indicator code'],
    var_name='Year', 
    value_name='Value'
)

In [ ]:
# Converto la colonna 'Year' in numerico

df_ed['Year'] = pd.to_numeric(df_ed['Year'], errors='coerce')

In [ ]:
df_ed.head()

In [ ]:
# Ordino il DataFrame per Paese, Indicatore e Anno e resetto l'indice

df_ed = df_ed.sort_values(by=['Country name', 'Indicator name', 'Year']).reset_index(drop=True)
df_ed.head()

In [ ]:
# Elimino la colonna 'Indicator code' in quanto non necessaria

df_ed = df_ed.drop(columns=['Indicator code'])

In [ ]:
df_ed.isna().sum()

In [ ]:
df_ed.info()

In [ ]:
# Creo una heatmap di disponibilità dei dati

plt.figure(figsize=(12, 8))
sns.heatmap(df_ed.pivot_table(index="Country name", 
                              columns="Year", 
                              values="Value")
            .isnull(), 
            cmap="viridis")
plt.title("Heatmap di disponibilità dei dati")
plt.xlabel("Anno")
plt.ylabel("Paese")
plt.show()

In [ ]:
# Creo un bar chart della media dei tre indicatori per ogni Paese

mean_values = df_ed.groupby(['Country name', 'Indicator name'])['Value'].mean().reset_index()
plt.figure(figsize=(12, 8))
sns.barplot(data=mean_values, x='Country name', y='Value', hue='Indicator name', errorbar=None)
plt.title("Media dei tre indicatori per ogni Paese")
plt.xlabel("Paese")
plt.ylabel("Media del valore")
plt.legend(title='Indicatore', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45) 
plt.show() 

In [ ]:
# Tengo solo i dati dal 2000 per un'analisi più completa

df_ed = df_ed[df_ed['Year'] >= 2000].reset_index(drop=True)

In [ ]:
# Creo una Facet Grid per visualizzare i trend degli indicatori nel tempo per ogni Paese

fig = px.line(
    df_ed,
    x="Year",
    y="Value",
    color="Indicator name",
    facet_col="Country name",
    facet_col_wrap=5,
    markers=True,
    title="Trend degli indicatori educativi (2000-2024)",
)

fig.update_layout(
    title_x=0.5,
    height=1200)
fig.show()

In [ ]:
# Creo un box plot per confrontare la distribuzione dei valori degli indicatori tra i Paesi

fig = px.box(
    df_ed,
    x="Indicator name",
    y="Value",
    color="Indicator name",
    points="all",   # mostra i punti, incluso gli outlier
    hover_data=["Country name", "Year"],
    title="Distribuzione dei valori degli indicatori educativi per Paese",
)

fig.update_layout(
    title_x=0.5,
    xaxis_title="Indicator",
    yaxis_title="Value",
    height=2500
)

fig.show()

In [ ]:
# Cerco gli outlier  per ogni indicatore con il metodo IQR, e stampo i nomi dei Paesi a cui appartengono

outlier_dict = {}
for indicator in indicators_list:
    df_indicator = df_ed[df_ed['Indicator name'] == indicator]
    Q1 = df_indicator['Value'].quantile(0.25)
    Q3 = df_indicator['Value'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_indicator[(df_indicator['Value'] < lower_bound) | (df_indicator['Value'] > upper_bound)]
    outlier_countries = outliers['Country name'].unique().tolist()
    outlier_dict[indicator] = outlier_countries

for indicator, countries in outlier_dict.items():
    print(f"Indicatore: {indicator}")
    print(f"Paesi con Outliers: {countries}")
    print()

## Analisi

Dall’analisi temporale degli indicatori emerge innanzitutto che il completion rate dell’istruzione secondaria è l’unico indicatore caratterizzato da un trend di crescita costante in tutti i Paesi considerati.
Questo andamento, spesso in controtendenza rispetto alla stagnazione osservata negli indicatori di expenditure e enrolment dei Paesi più sviluppati, 
suggerisce che i progressi in termini di completamento degli studi non dipendono esclusivamente dall’aumento degli investimenti o dei tassi di iscrizione.

Paesi come Brasile e Cina mostrano le crescite più marcate nel completion rate; tuttavia non risulta possibile mettere in correlazione tali imponenti crescite con una tendenza simile nell'impegno governativo in termini di spesa nel settore educativo.
Nel caso della Cina, inoltre, la crescita avviene nonostante l’assenza quasi totale di dati sull’enrolment, il che indica che i meccanismi di raccolta dati e trasparenza possono essere molto diversi e contribuire alla classificazione del Paese come outlier statistico più che sostanziale.

I Paesi che risultano più frequentemente outlier — Sud Africa, India e Kenya — presentano specificità socio-economiche e strutturali che possono giustificare tali deviazioni. 
Il Sud Africa mostra valori carenti e discontinui nel completion rate, probabilmente legati a forti disuguaglianze interne.
L’India, che risulta outlier negli indicatori di enrolment e completion, registra oscillazioni legate alla complessità territoriale e alla forte eterogeneità regionale.
Il Kenya si distingue come outlier sia nell’enrolment sia nell’expenditure: l’aumento delle spese educative in determinati anni, seguito da un taglio netto, genera un andamento frastagliato che si riflette direttamente nel tasso di iscrizione. Tuttavia, il fatto che il Paese risulti outlier positivo nell’expenditure indica un impegno significativo in certi periodi, impegno che si traduce in un temporaneo incremento dell’enrolment prima della successiva contrazione.

In generale, ciò che rende questi Paesi outlier è la combinazione di instabilità economica, irregolarità nei finanziamenti, differenze nella qualità delle infrastrutture scolastiche e nella capacità di raccolta dati, che produce traiettorie atipiche rispetto al gruppo.

In [ ]:
# Espando la ricerca caricando dataset su mortalità infantile, GDP pro capite e aspettativa di vita

In [ ]:
# Carico il primo dataset sul GDP pro capite

df_gdp = pd.read_csv('/Users/valerioquaranta/Documents/Data Manipulation and Visualization/Progetto/Prog_corr/current_GDP_percap.csv', sep=',', skiprows=4) # Salto le prime 4 righe di metadati

In [ ]:
df_gdp.head()

In [ ]:
# Pulsco i nomi delle colonne ed elimino quelle non necessarie

df_gdp.columns = df_gdp.columns.str.strip()
df_gdp = df_gdp.drop(columns=['Indicator Code', 'Unnamed: 69'])
df_gdp.head()

In [ ]:
# Filtro il DataFrame per includere solo i Paesi di interesse, lo converto in formato long, converto la colonna 'Year' in numerico, rimuovo i valori NaN e tengo solo i dati dal 2000 in poi

df_gdp = df_gdp[df_gdp['Country Name'].isin(countries_list)]
df_gdp = df_gdp.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name'],
    var_name='Year',
    value_name='GDP per Capita'
).reset_index(drop=True).drop(columns=['Indicator Name'])
df_gdp['Year'] = pd.to_numeric(df_gdp['Year'], errors='coerce')
df_gdp = df_gdp.dropna(subset=['GDP per Capita'])
df_gdp = df_gdp[df_gdp['Year'] >= 2000].reset_index(drop=True)
df_gdp.head()

In [ ]:
# Ordino il DataFrame per Paese e Anno e approssimo i valori del GDP pro capite

df_gdp = df_gdp.sort_values(by=['Country Name', 'Year']).reset_index(drop=True)
df_gdp['GDP per Capita'] = df_gdp['GDP per Capita'].round(2)
df_gdp.head()

In [ ]:
# Carico il secondo dataset sull'aspettativa di vita, come file csv con virgole come separatori

df_life = pd.read_csv('/Users/valerioquaranta/Documents/Data Manipulation and Visualization/Progetto/Prog_corr/life_expct.csv', sep=',')

In [ ]:
df_life.head()

In [ ]:
# Pulisco i nomi delle colonne ed elimino quelle non necessarie

df_life.columns = df_life.columns.str.strip()
df_life = df_life.drop(columns=['FREQ', 'FREQ_LABEL', 'INDICATOR', 'UNIT_MEASURE', 'UNIT_MEASURE_LABEL', 'AGG_METHOD', 'AGG_METHOD_LABEL', 'DECIMALS', 'DECIMALS_LABEL', 'DATABASE_ID', 'DATABASE_ID_LABEL', 'DATABASE_ID_LABEL', 'UNIT_MULT_LABEL', 'DATA_SOURCE', 'DATA_SOURCE_LABEL', 'OBS_STATUS', 'OBS_STATUS_LABEL', 'OBS_CONF', 'OBS_CONF_LABEL', 'UNIT_MULT'])
df_life = df_life.rename(columns={
    'REF_AREA': 'Country Code',
    'REF_AREA_LABEL': 'Country Name',
})
df_life.head()

In [ ]:
# Filtro il DataFrame per includere solo i Paesi di interesse, lo converto in formato long, converto la colonna 'Year' in numerico, rimuovo i valori NaN e tengo solo i dati dal 2000 in poi

df_life = df_life[df_life['Country Name'].isin(countries_list)]
df_life = df_life.melt(
    id_vars=['Country Name', 'Country Code'],
    var_name='Year',
    value_name='Life Expectancy (Years)'
).reset_index(drop=True)
df_life['Year'] = pd.to_numeric(df_life['Year'], errors='coerce')
df_life = df_life.dropna(subset=['Life Expectancy (Years)'])
df_life = df_life[df_life['Year'] >= 2000].reset_index(drop=True)
df_life.head()

In [ ]:
# Ordino il DataFrame per Paese e Anno e approssimo i valori dell'aspettativa di vita

df_life = df_life.sort_values(by=['Country Name', 'Year']).reset_index(drop=True)
df_life['Life Expectancy (Years)'] = df_life['Life Expectancy (Years)'].round(2)
df_life.head()

In [ ]:
# Correggo la colonna 'Year' 

df_life['Year'] = df_life['Year'].astype('Int64')
df_life.head()

In [ ]:
# Carico il terzo dataset sulla mortalità infantile

df_mort = pd.read_csv('/Users/valerioquaranta/Documents/Data Manipulation and Visualization/Progetto/Prog_corr/infant_mortality.csv', sep=',')

In [ ]:
df_mort.head()

In [ ]:
# Vedo i valori dell acolonna INDICATOR_LABEL

df_mort['INDICATOR_LABEL'].unique()

In [ ]:
df_mort.info()

In [ ]:
# A partire dal dataframe precdente, ne creo uno con le sole colonne necessarie, cioe: REF_AREA, REF_AREA_LABEL e TIME_PERIOD

df_mort = df_mort.rename(columns={
    'REF_AREA': 'Country Code',
    'REF_AREA_LABEL': 'Country Name',
    'INDICATOR_LABEL': 'Indicator Name',
    'TIME_PERIOD': 'Year',
    'OBS_VALUE': 'Infant Mortality Rate (per 1000 live births)'
})
df_mort = df_mort[['Country Code', 'Country Name', 'Year', 'Infant Mortality Rate (per 1000 live births)']]
df_mort.head()



In [ ]:
# Filtro il DataFrame per includere solo i Paesi di interesse, converto la colonna 'Year' in numerico, rimuovo i valori NaN e tengo solo i dati dal 2000 in poi

df_mort = df_mort[df_mort['Country Name'].isin(countries_list)]
df_mort['Year'] = pd.to_numeric(df_mort['Year'], errors='coerce')
df_mort = df_mort.dropna(subset=['Infant Mortality Rate (per 1000 live births)'])
df_mort = df_mort[df_mort['Year'] >= 2000].reset_index(drop=True)
df_mort.head()

In [ ]:
# Ordino il DataFrame per Paese e Anno e approssimo i valori della mortalità infantile

df_mort = df_mort.sort_values(by=['Country Name', 'Year']).reset_index(drop=True)
df_mort['Infant Mortality Rate (per 1000 live births)'] = df_mort['Infant Mortality Rate (per 1000 live births)'].round(2)
df_mort.head()

In [ ]:
# Adesso creo un unico dataframe unendo i quattro dataframes precedenti
# Prima di tutto modifico il df_ed per avere una colonna per ogni indicatore

indicators_map = {
    "Completion rate, upper secondary education, both sexes (%)": "Secondary Completion Rate (%)",
    "Total net enrolment rate, primary, both sexes (%)": "Primary Enrolment Rate(%)",
    "Government expenditure on education as a percentage of GDP (%)": "Education Expenditure (% of GDP)",
}

df_ed = df_ed.pivot_table(
    index=['Country name', 'Country code', 'Year'],
    columns='Indicator name',
    values='Value'
).reset_index()
df_ed = df_ed.rename(columns=indicators_map)
df_ed.head()

In [ ]:
# Arrotondo i valori degli indicatori educativi

df_ed['Primary Enrolment Rate(%)'] = df_ed['Primary Enrolment Rate(%)'].round(2)
df_ed['Secondary Completion Rate (%)'] = df_ed['Secondary Completion Rate (%)'].round(2)
df_ed['Education Expenditure (% of GDP)'] = df_ed['Education Expenditure (% of GDP)'].round(2)
df_ed.head()

In [ ]:
# Rinomino le colonne per uniformità

df_ed = df_ed.rename(columns={
    'Country name': 'Country Name',
    'Country code': 'Country Code'
})
df_ed.head()

In [ ]:
# Unisco i quattro dataframes

df_final = pd.merge(df_ed, df_gdp, on=['Country Name', 'Country Code', 'Year'], how='inner')
df_final = pd.merge(df_final, df_life, on=['Country Name', 'Country Code', 'Year'], how='inner')
df_final = pd.merge(df_final, df_mort, on=['Country Name', 'Country Code', 'Year'], how='inner')
df_final.head()

In [ ]:
df_final.info()

In [ ]:
# cerco quanti valori NaN sono presenti nel dataframe finale

df_final.isna().sum()

In [ ]:
# Converto la colonna 'Life Expectancy (Years)' in numerico

df_final['Life Expectancy (Years)'] = (
    df_final['Life Expectancy (Years)']
    .astype(str)
    .str.replace(",", ".", regex=False)   # se ci fossero virgole
    .str.extract(r"(\d+\.?\d*)")[0]       # estrae solo numeri
    .astype(float)
)
df_final.info()

In [ ]:
# Completo i valori NaN con l'interpolazione lineare per ogni Paese

cols = [
    'Primary Enrolment Rate(%)',
    'Secondary Completion Rate (%)',
    'Education Expenditure (% of GDP)',
    'GDP per Capita',
    'Life Expectancy (Years)',
    'Infant Mortality Rate (per 1000 live births)'
]

df_final = df_final.sort_values(by=['Country Name', 'Year']).reset_index(drop=True)

df_final[cols] = (
    df_final.groupby('Country Name')[cols]
            .transform(lambda group: group.interpolate(method='linear')) # uso transform per mantenere l'indice
)


In [ ]:
df_final.isna().sum()

In [ ]:
# Uso ffill e bfill per completare i valori NaN rimanenti

df_final[cols] = df_final.groupby('Country Name')[cols].ffill().bfill()

In [ ]:
df_final.isna().sum()

In [ ]:
df_final.head()

In [ ]:
# Ora creo una heatmap per visualizzare le correlazioni tra le variabili del dataframe finale

numeric_cols = [
    "Secondary Completion Rate (%)",
    "Education Expenditure (% of GDP)",
    "Primary Enrolment Rate(%)",
    "GDP per Capita",
    "Life Expectancy (Years)",
    "Infant Mortality Rate (per 1000 live births)"
]

corr_matrix = df_final[numeric_cols].corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale="RdBu_r",
    zmin=-1,
    zmax=1,
    title="Correlation Heatmap of Key Indicators",
)

fig.update_layout(
    title_x=0.5,
    width=1200,  
    height=900,
    font=dict(size=16),
    margin=dict(l=60, r=60, t=80, b=60)
)

fig.update_coloraxes(cmid=0)

fig.update_traces(
    texttemplate="%{z:.2f}",
    textfont=dict(size=14, color="white")
)

fig.show()

## Analisi delle correlazioni tra istruzione, sviluppo e indicatori socio-demografici

L’analisi delle correlazioni mette in evidenza una rete di relazioni molto coerenti tra variabili educative, economiche e demografiche. In particolare, il Secondary Completion Rate risulta l’indicatore più fortemente associato sia alle dimensioni economiche che a quelle legate alla salute e allo sviluppo umano. La correlazione molto alta con il GDP per capita (0.87) suggerisce che livelli più elevati di completamento dell’istruzione secondaria si osservano tipicamente nei Paesi con economie più strutturate e stabili, dove l’accesso all’istruzione è sostenuto da investimenti pubblici, infrastrutture e un migliore benessere generale. Coerentemente, la correlazione positiva con la Life Expectancy (0.81) e quella fortemente negativa con l’Infant Mortality Rate (-0.85) indicano che sistemi educativi più efficaci si associano a condizioni di salute migliori e a una più bassa vulnerabilità sanitaria.

Il Primary Enrolment Rate mostra un comportamento simile: presenta una correlazione elevata con la Life Expectancy (0.80) e una correlazione negativa con la mortalità infantile (-0.78). Questo rafforza l’idea che l’accesso all’istruzione primaria rappresenti una componente essenziale dello sviluppo umano e che i Paesi in cui la scolarizzazione primaria è più diffusa siano anche quelli con migliori risultati in termini di salute pubblica e benessere della popolazione. La relazione moderata con il GDP per capita (0.50) conferma inoltre che l’accesso iniziale alla scuola è già un indicatore sensibile del livello di sviluppo economico di un Paese.

Un risultato interessante riguarda invece la Education Expenditure (% of GDP), la cui correlazione con gli altri indicatori è sorprendentemente debole. L’investimento percentuale sul PIL mostra solo una lieve correlazione positiva con il completion rate (0.30) e con il GDP per capita (0.27), mentre risulta quasi del tutto scollegato dagli indicatori sanitari e dall’enrolment primario. Questo suggerisce che la percentuale di spesa relativa al PIL non è di per sé un indicatore affidabile della qualità dei risultati educativi: Paesi con PIL più basso potrebbero dedicare una quota maggiore alle spese educative, senza però garantire risultati comparabili, mentre Paesi ad alto PIL potrebbero investire una quota più contenuta ma in valori assoluti molto più elevati. Il dato sembra quindi confermare la necessità di interpretare la spesa in istruzione non solo come percentuale del PIL, ma anche nel contesto di fattori come efficienza gestionale, capacità amministrativa e distribuzione della spesa.

Le correlazioni tra indicatori economici e sanitari — in particolare quella molto alta tra Life Expectancy e Infant Mortality Rate (-0.94) — mostrano la coerenza interna del dataset e confermano l’affidabilità delle relazioni individuate: Paesi più ricchi, più istruiti e con migliori performance educative tendono a presentare anche migliori condizioni di salute e livelli inferiori di mortalità infantile.

Nel complesso, questa matrice di correlazioni suggerisce un quadro chiaro: istruzione, salute e sviluppo economico sono dimensioni profondamente interconnesse, e i Paesi che investono in un accesso efficace e continuativo all’istruzione — soprattutto secondaria — raccolgono benefici evidenti in termini di prosperità economica e benessere sociale. Le variabili educative analizzate non agiscono isolatamente, ma si integrano in un sistema più ampio di fattori che definiscono il livello di sviluppo di un Paese e le opportunità delle future generazioni.

In [ ]:
# Creo un bubble chart animato per visualizzare la relazione tra GDP pro capite, aspettativa di vita e tasso di completamento dell'istruzione secondaria nel tempo
# Per ottenere questa vsualizzazione, modifico il df_final per tenere solo i dati fino al 2020

df_viz = df_final[df_final['Year'] <= 2020].reset_index(drop=True)

fig = px.scatter(
    df_viz,
    x="GDP per Capita",
    y="Life Expectancy (Years)",
    size="Secondary Completion Rate (%)",
    color="Country Name",
    hover_name="Country Name",
    animation_frame="Year",
    log_x=True,
    size_max=60,
    range_x=[300, 100000],
    range_y=[50, 90],
    title="GDP per Capita vs Life Expectancy with Secondary Completion Rate (2000-2020)"
)

fig.update_layout(
    title_x=0.5,
    height=700)
fig.show()



In [ ]:
# Creo un altr0 bubble chart animato per vedere la relazione tra tasso di completmento secondario, aspettativa di vita e gdp pro capite nel tempo

fig = px.scatter(
    df_viz,
    x="Secondary Completion Rate (%)",
    y="Life Expectancy (Years)",
    size="GDP per Capita",
    color="Country Name",
    hover_name="Country Name",
    animation_frame="Year",
    size_max=60,
    range_x=[20, 100],
    range_y=[50, 90],
    title="Secondary Completion Rate vs Life Expectancy with GDP per Capita (2000-2020)"
)
fig.update_layout(
    title_x=0.5,
    height=700)
fig.show()

In [ ]:
# Vedo le statistiche descrittive 

df_final.describe()

In [ ]:
# Identifico gli outlier per ogni indicatore usando il metodo IQR sul dataframe finale, e stampo i nomi dei Paesi a cui appartengono

outlier_dict_final = {}
for col in cols:
    Q1 = df_final[col].quantile(0.25)
    Q3 = df_final[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_final[(df_final[col] < lower_bound) | (df_final[col] > upper_bound)]
    outlier_countries = outliers['Country Name'].unique().tolist()
    outlier_dict_final[col] = outlier_countries

for indicator, countries in outlier_dict_final.items():
    print(f"Indicatore: {indicator}")
    print(f"Paesi con Outliers: {countries}")
    print()

# Conclusioni

### L’analisi congiunta dei sei indicatori educativi, economici e sanitari ha mostrato come i sistemi educativi si inseriscano in un ecosistema più ampio, dove sviluppo economico e condizioni di salute evolvono in modo fortemente correlato con i risultati scolastici. Le statistiche descrittive evidenziano valori medi coerenti con i trend globali, ma anche differenze strutturali importanti tra Paesi, che emergono con chiarezza dagli outlier e dai casi studio più estremi.

1. Investire sull’istruzione secondaria genera benefici trasversali (alta robustezza analitica)

La correlazione elevata tra Completion Rate secondario e variabili macro-economiche e sanitarie — in particolare GDP pro capite (0.87), aspettativa di vita (0.81) e mortalità infantile (–0.94) — indica che il rafforzamento dell’istruzione secondaria dovrebbe rappresentare una priorità strategica nei Paesi a medio reddito.
Raccomandazione: promuovere politiche di completamento della scuola secondaria, poiché gli effetti positivi superano l’ambito educativo e accelerano il miglioramento degli indicatori demografici e sanitari.

2. La qualità della spesa pubblica in educazione pesa più della quantità

La correlazione tra spesa pubblica in educazione (% GDP) e gli outcome è debole o nulla. Anzi, Paesi come Kenya e Sud Africa spendono sopra la media mondiale (4.76% del PIL), ma non ottengono miglioramenti proporzionati nei principali indicatori educativi.
Al contrario, la Cina, pur posizionandosi nella fascia bassa della spesa, mostra la crescita più significativa in completamento secondario, aspettativa di vita e riduzione della mortalità infantile.
Raccomandazione: concentrare gli investimenti su interventi ad alto ritorno (qualità dell’insegnamento, infrastrutture essenziali), piuttosto che aumentare la quota del PIL senza una revisione dell’efficienza allocativa.

3. Ampliare l’istruzione primaria genera effetti cumulativi su tutti gli indicatori successivi

Il Primary Enrolment (media 95.9%, con range ampio: 64.9–99.9%) mostra correlazioni molto elevate con aspettativa di vita (0.80) e moderate con il completamento secondario (0.56).
Questo indica che le basi costruite nei primi cicli scolastici determinano la traiettoria formativa e di salute della popolazione nei decenni successivi.
Raccomandazione: rafforzare le politiche di accesso alla scuola primaria nelle aree con iscrizione discontinua o fragile (India, Kenya, Sud Africa), incluse campagne di sostegno alle famiglie, programmi nutrizionali e riduzione dei costi indiretti dell’educazione.

5. Aumentare l’aspettativa di vita e ridurre la mortalità infantile passa anche dall’educazione

La correlazione più forte di tutto il dataset (–0.94 tra mortalità infantile e aspettativa di vita) mostra come i due indicatori sanitari siano strettamente intrecciati.
Tuttavia, il dato chiave è che entrambi migliorano sostanzialmente nei Paesi che incrementano la qualità e la continuità dell’istruzione, specialmente primaria e secondaria.
Raccomandazione: adottare strategie che intregano salute ed educazione, in particolare nei Paesi a basso reddito.

6. Considerare la dimensione temporale: il miglioramento medio richiede almeno un decennio

I dati, distribuiti tra 2000 e 2023, mostrano che gli indicatori migliorano gradualmente:

Progressione media annua nel Completion Rate ~ +0.7%

Incrementi lenti ma stabili dell’aspettativa di vita

Calo della mortalità infantile più marcato nei Paesi a crescita economica sostenuta
Raccomandazione: le policy devono essere mantenute nel tempo; i ritorni sugli investimenti educativi emergono tipicamente dopo cicli strutturali di 8–12 anni.